# La librairie `scikit-learn`

Matériel de cours **optionnel** rédigé par Pascal Germain, 2018
*********************

Autres références sur `scikit-learn`:
* https://scikit-learn.org/stable/user_guide.html
* http://scipy-lectures.org/packages/scikit-learn/index.html
---------

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
repertoire_mnist = '../data/mnist/' # Modifier le répertoire au besoin

Pour commencer, nous travaillerons dans un contexte de *classification binaire*, en apprenant à distinguer des images de `2` et des images `3`. Chargeons les données et créons l'ensemble d'apprentissage.

In [ ]:
def charger_mnist(repertoire, etiquettes=None, max_par_etiquettes=None):
    if etiquettes is None:
         etiquettes = np.arange(10)
    images_list = [None] * len(etiquettes)
    labels_list = [None] * len(etiquettes)
    for i, val in enumerate(etiquettes):
        nom_fichier = repertoire + f'mnist_{val}.gz'
        images_list[i] = np.genfromtxt(nom_fichier, max_rows=max_par_etiquettes, dtype=np.float32)
        nb = images_list[i].shape[0]

        labels_list[i] = i*np.ones(nb, dtype=np.int64)
        print(val, ':', nb, 'images')
        
    x = np.vstack(images_list)
    y = np.concatenate(labels_list)
    print('Total :', len(y), 'images')
    return x, y

In [ ]:
data_x, data_y = charger_mnist(repertoire_mnist, etiquettes=[2,3], max_par_etiquettes=None) 

In [ ]:
print('data_x:', data_x.shape)
print('data_y:', data_y.shape)

In [ ]:
plt.hist(data_x.reshape(-1)); # Histogramme des valeurs des pixels dans tout le dataset

In [ ]:
data_y

In [ ]:
im2 = data_x[0,:].reshape(28,28)
plt.imshow(im2, cmap=plt.cm.gray)
plt.colorbar()

In [ ]:
im3 = data_x[-1,:].reshape(28,28)
plt.imshow(im3, cmap=plt.cm.gray)
plt.colorbar()

### Séparation des données en un ensemble d'apprentissage et un ensemble de test

Voir: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=10000, random_state=42)
print('train_x:', train_x.shape)
print('test_x:', test_x.shape)
print('train_y:', train_y.shape)
print('test_y:', test_y.shape)

In [ ]:
train_y[0:100]

### Algorithme d'apprentissage (Exemple de la régression logistique)

Voir: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
from sklearn.linear_model import LogisticRegression

**Étape 1:** Initialiser l'algorithme d'apprentissage (constructeur de la *classe*).

In [ ]:
LogisticRegression?

In [ ]:
predicteur = LogisticRegression(C=.1)

**Étape 2:** Exécuter l'algorithme sur les données d'apprentissage (méthode `fit`).

In [ ]:
predicteur.fit?

In [ ]:
predicteur.fit(train_x, train_y)

**Étape 3:** Prédire sur des nouvelles données (méthode `predict`).

In [ ]:
predicteur.predict?

In [ ]:
predictions = predicteur.predict(test_x[0:100,:])
predictions

*Regardons quelques erreurs de classifications*

In [ ]:
echecs = np.nonzero(predicteur.predict(test_x[0:100,:]) != test_y[0:100])
echecs

In [ ]:
fig, axes = plt.subplots(1, len(echecs[0]), figsize=(10, 4))
for i, ax in zip(echecs[0], axes):
    ax.imshow(test_x[i].reshape(28,28), cmap=plt.cm.gray)

### Évaluer la performance d'un algorithme d'apprentissage

Voir: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score?

In [ ]:
predicteur = LogisticRegression(C=1)
predicteur.fit(train_x, train_y)
train_pred = predicteur.predict(train_x)
test_pred = predicteur.predict(test_x)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

Essayons avec un autre paramètre $C$.

In [ ]:
predicteur = LogisticRegression(C=.1)
predicteur.fit(train_x, train_y)
train_pred = predicteur.predict(train_x)
test_pred = predicteur.predict(test_x)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

### Pré-traitement des données

Voir: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
StandardScaler?

In [ ]:
scaler = StandardScaler()
scaler.fit(train_x)
train_x_prime = scaler.transform(train_x)
test_x_prime = scaler.transform(test_x)

In [ ]:
i = 1
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
cax = axes[1].imshow(train_x[i].reshape(28,28), cmap=plt.cm.gray)
fig.colorbar(cax, ax=axes[1])
cax = axes[0].imshow(train_x_prime[i].reshape(28,28), cmap=plt.cm.gray)
fig.colorbar(cax, ax=axes[0]);

In [ ]:
predicteur = LogisticRegression(C=0.1)
predicteur.fit(train_x_prime, train_y)
train_pred = predicteur.predict(train_x_prime)
test_pred = predicteur.predict(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

### Sélections de modèle par validation croisée

Voir: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
C_space = np.logspace(-4,4,10)
parametres = {'C':C_space}
parametres

In [ ]:
predicteur = LogisticRegression()
predicteur_CV = GridSearchCV(predicteur, parametres, n_jobs=4, cv=5, return_train_score=False)
predicteur_CV.fit(train_x_prime, train_y)

In [ ]:
train_pred = predicteur_CV.predict(train_x_prime)
test_pred = predicteur_CV.predict(test_x_prime)

print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))
print('Paramètres sélectionnés par la validation croisée :', predicteur_CV.best_params_)

In [ ]:
cv_results = predicteur_CV.cv_results_
predicteur_CV.cv_results_.keys()

In [ ]:
from pandas import DataFrame
DataFrame(cv_results)

In [ ]:
plt.semilogx(cv_results['param_C'], cv_results['mean_test_score'])

### Essayons avec un autre algorithme d'apprentissage (SVM)

Voir: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
from sklearn.svm import SVC

In [ ]:
SVC?

In [ ]:
predicteur = SVC(C=1.0, gamma='auto')
predicteur.fit(train_x_prime, train_y)
train_pred = predicteur.predict(train_x_prime)
test_pred = predicteur.predict(test_x_prime)
accuracy_score(train_y, train_pred) , accuracy_score(test_y, test_pred)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))

In [ ]:
# ATTENTION: L'exécution de cette cellule de code devrait prendre un certain temps...
C_space = np.logspace(-3, 3, 6)
gamma_space = np.logspace(-5, -1, 6)
parametres = {'C':C_space, 'gamma':gamma_space}

predicteur = SVC()
predicteur_CV = GridSearchCV(predicteur, parametres, n_jobs=4, cv=3, return_train_score=False)
predicteur_CV.fit(train_x_prime, train_y)

train_pred = predicteur_CV.predict(train_x_prime)
test_pred = predicteur_CV.predict(test_x_prime)
print('Précision train:', accuracy_score(train_y, train_pred) )
print('Précision test :', accuracy_score(test_y, test_pred))
print('Paramètres sélectionnés par la validation croisée :', predicteur_CV.best_params_)

In [ ]:
cv_results = predicteur_CV.cv_results_
DataFrame(cv_results)

In [ ]:
nb_gamma = len(gamma_space)
nb_C = len(C_space)
plt.matshow(cv_results['mean_test_score'].reshape((nb_C,nb_gamma)), cmap='Blues') 
plt.xticks(np.arange(nb_gamma), ["%.5f" % g for g in gamma_space], rotation=90)
plt.yticks(np.arange(nb_C), ["%.3f" % c for c in C_space])
plt.colorbar()